In [1]:
from ImportLocalData import loadData

In [2]:
from BalanceClassDistribution import AdjustClassSamples, NumberOfSamplesClass

In [ ]:
# Import one of the custom STKG files
# Call BalanceClass... to handle outliers if you need
# Please change path names based on your local files 
data = loadData('/home/gozde/GEREKLI_DOSYALAR/STKGNN_Codes_Data (4)/STKGNN_Codes_Data/STKG_ LocalData/MS-STKG-Medium/node_features.txt', '/home/gozde/GEREKLI_DOSYALAR/STKGNN_Codes_Data (4)/STKGNN_Codes_Data/STKG_ LocalData/MS-STKG-Medium/edges.txt', '/home/gozde/GEREKLI_DOSYALAR/STKGNN_Codes_Data (4)/STKGNN_Codes_Data/STKG_ LocalData/MS-STKG-Medium/edge_features.txt', '/home/gozde/GEREKLI_DOSYALAR/STKGNN_Codes_Data (4)/STKGNN_Codes_Data/STKG_ LocalData/MS-STKG-Medium/node_labels.txt')
data = AdjustClassSamples(data) #this is optinal, yet in papr we used

Import required packages

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch.nn import Dropout
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, top_k_accuracy_score
from torch_geometric.utils import subgraph, add_self_loops, degree
import numpy as np
from torch_geometric.data import Data

STIP_GCN Model Structure

In [7]:
# We implemented "STIP-GCN:Space-time interest points graph convolutional network for action recognition" 
#study for comparatve anaysis with our dataset and report it through top-1 / top-5 accuracy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# STIP-GCN MODEL WHICH COMPATIBLE our STKGs daTaset
class STIP_GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, edge_attr_dim):
        super(STIP_GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        # A small MLP to learn edge weights from edge features
        self.edge_weight_net = torch.nn.Sequential(
            torch.nn.Linear(edge_attr_dim, 1),
            torch.nn.Sigmoid())

        self.fc = torch.nn.Linear(hidden_channels, out_channels)
        self.dropout = Dropout(p=0.5)
        self.alpha = torch.nn.Parameter(torch.tensor(0.5))  # Initialized at 0.5 for equal contribution

    def forward(self, data):
        learned_weight = self.edge_weight_net(data.edge_attr).squeeze()
# Split edge features into coordinates of source and destination nodes
        r_i = data.edge_attr[:, :3]  # Source node coordinates
        r_j = data.edge_attr[:, 3:]  # Target node coordinates
        h_i = data.x[data.edge_index[0]]
        h_j = data.x[data.edge_index[1]]

        cos_sim = F.cosine_similarity(h_i, h_j)
        similarity = (cos_sim + 1) / 2
# euclidean distance between of nodes
        distance = torch.norm(r_i - r_j, p=2, dim=1)
        distance = torch.exp(-distance)
# structural weight using similarity and distance
        structural_weight = similarity / distance
# remove weights below the mean value
        structural_weight = (structural_weight > structural_weight.mean()).float() * structural_weight
        edge_w = self.alpha * learned_weight + (1 - self.alpha) * structural_weight
        edge_in, edge_w = add_self_loops(data.edge_index, edge_w, num_nodes=data.num_nodes)
# normalizon edge weights for spectral GCN
        row, col = edge_in
        deg = degree(row, data.num_nodes, dtype=data.x.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        norm = deg_inv_sqrt[row] * edge_w * deg_inv_sqrt[col]

        x = F.relu(self.conv1(data.x, edge_in, norm))
        x = self.dropout(x)
        x = F.relu(self.conv2(x, edge_in, norm))
        return self.fc(x)


Train Function Definition for the Model

In [18]:
def train_model(model, data, train_idx, val_idx, test_idx, model_name="STIP_GCNmodel"):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    loss_fn = torch.nn.CrossEntropyLoss()
    bestVal_acc = 0
    bestVal_top5 = 0
    bestModel_state = None

    for epoch in range(200):  # Number of epoch defined based on paper
        model.train()
        optimizer.zero_grad()
        out = model(data)
        loss = loss_fn(out[train_idx], data.y[train_idx])
        if torch.isnan(loss):
            print("NaN loss detected!")
            break

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        # Evaluation every 10 epochs
        if (epoch + 1) % 10 == 0:
            model.eval()
            with torch.no_grad():
                outEval = model(data)
                pred = outEval.argmax(dim=1)
# Top-1 and Top-5 acuracy for training and valdation sets
                train_acc = accuracy_score(data.y[train_idx].cpu().numpy(), pred[train_idx].cpu().numpy())
                train_top5 = top_k_accuracy_score(
                    data.y[train_idx].cpu().numpy(),outEval[train_idx].cpu().numpy(), k=5,labels=np.arange(outEval.size(1)))
                val_acc = accuracy_score(data.y[val_idx].cpu().numpy(), pred[val_idx].cpu().numpy())
                val_top5 = top_k_accuracy_score(data.y[val_idx].cpu().numpy(),
                    outEval[val_idx].cpu().numpy(),k=5,
                    labels=np.arange(outEval.size(1)))
                if val_acc > bestVal_acc:
                    bestVal_acc = val_acc
                    bestVal_top5 = val_top5
                    bestModel_state = model.state_dict()

                print(f'Model: {model_name}, Epoch: {epoch+1}, Loss: {loss.item():.4f}, 'f'Train Acc: {train_acc:.4f}, Train Top-5: {train_top5:.4f}, 'f'Val Acc: {val_acc:.4f}, Val Top-5: {val_top5:.4f}')

    if bestModel_state is not None:
        model.load_state_dict(bestModel_state)
    model.eval()
    with torch.no_grad():
        outTest = model(data)
        predTest = outTest.argmax(dim=1)
        test_acc = accuracy_score(data.y[test_idx].cpu().numpy(), predTest[test_idx].cpu().numpy())
        test_top5 = top_k_accuracy_score(data.y[test_idx].cpu().numpy(),
            outTest[test_idx].cpu().numpy(),k=5,labels=np.arange(outTest.size(1)))

    print(f'\nSTIP_GCN Moel: {model_name}, Final Test Top-1 Acuracy: {test_acc*100:.2f}%')
    print(f'STIP_GCN Model: {model_name}, Final Test Top-5 Accuracy: {test_top5*100:.2f}%')

    return test_acc, test_top5


Evaluation of the Model

In [ ]:
if __name__ == '__main__':
    data = data.to(device)
    scaler = StandardScaler()
    data.x = scaler.fit_transform(data.x.cpu().numpy())
    data.x = torch.tensor(data.x, dtype=torch.float).to(device)

# Split dataset into 80% train, 10% validation, 10% test by stratfied sampling
    indices = np.arange(data.num_nodes)
    train_idx, test_val_idx = train_test_split(indices, test_size=0.2, stratify=data.y.cpu().numpy(), random_state=42)
    val_idx, test_idx = train_test_split(test_val_idx, test_size=0.5, stratify=data.y[test_val_idx].cpu().numpy(), random_state=42)
    train_idx = torch.tensor(train_idx, dtype=torch.long, device=device)
    val_idx = torch.tensor(val_idx, dtype=torch.long, device=device)
    test_idx = torch.tensor(test_idx, dtype=torch.long, device=device)
# Initalize STIP-GCN model with hyperparameters
    modelCombined = STIP_GCN(in_channels=data.x.size(1),hidden_channels=256, out_channels=len(data.y.unique()),edge_attr_dim=data.edge_attr.size(1)).to(device)
    
    print("\nTraining Paper Based STIP-GCN Model Comptiple with our STKGs datset")
    test_acc_combined, test_top5_combined = train_model(modelCombined, data, train_idx, val_idx, test_idx, "STIP_GCN")